In [4]:
import os

output_dir = "data_input/basemap_elements"
os.makedirs(output_dir, exist_ok=True)


In [8]:
natural_earth_layers = [
    # Catégorie Cultural
    ("cultural", "admin_0_countries"),
    ("cultural", "admin_0_boundary_lines_land"),
    ("cultural", "admin_0_countries_lakes"),
    ("cultural", "admin_0_sovereignty"),
    ("cultural", "admin_1_states_provinces"),
    ("cultural", "admin_1_states_provinces_lakes"),
    ("cultural", "admin_1_states_provinces_lines"),
    ("cultural", "populated_places"),
    ("cultural", "urban_areas"),
    ("cultural", "roads"),
    ("cultural", "railroads"),
    # Catégorie Physical
    ("physical", "land"),
    ("physical", "ocean"),
    ("physical", "minor_islands"),
    ("physical", "reefs"),
    ("physical", "lakes"),
    ("physical", "rivers_lake_centerlines"),
    ("physical", "glaciated_areas"),
    ("physical", "antarctic_ice_shelves_polys"),
    ("physical", "antarctic_ice_shelves_lines"),
    ("physical", "coastline"),
    ("physical", "geographic_lines"),
    ("physical", "graticules_1"),
    ("physical", "graticules_5"),
    ("physical", "graticules_10"),
]
# Résolutions : "10m", "50m", "110m"


# Function

In [11]:
import os
import requests
import shutil
from zipfile import ZipFile
from pathlib import Path
import geopandas as gpd

def download_and_convert_natural_earth(layers, resolution="10m", output_dir="data_input/basemap_elements"):
    """
    Télécharge, charge, exporte en GPKG, et nettoie pour une liste de couches Natural Earth.
    On ne garde que le .gpkg final.
    """
    os.makedirs(output_dir, exist_ok=True)
    base_url_tpl = "https://naturalearth.s3.amazonaws.com/{res}_{cat}/ne_{res}_{name}.zip"

    for category, name in layers:
        url = base_url_tpl.format(res=resolution, cat=category, name=name)
        zip_path = Path(output_dir) / f"ne_{resolution}_{name}.zip"
        extract_path = Path(output_dir) / f"ne_{resolution}_{name}"
        gpkg_path = Path(output_dir) / f"ne_{resolution}_{name}.gpkg"

        print(f"\n🔻 Traitement: {name} ({category}, {resolution})")
        
        # 1. Télécharger si pas déjà là
        if not zip_path.exists():
            r = requests.get(url)
            if r.status_code != 200:
                print(f"  ❌ Erreur: {r.status_code} pour {url}")
                continue
            with open(zip_path, "wb") as f:
                f.write(r.content)
            print("  ✅ ZIP téléchargé.")
        else:
            print("  ⚡ ZIP déjà présent.")

        # 2. Dézipper
        if not extract_path.exists():
            with ZipFile(zip_path, "r") as zip_ref:
                zip_ref.extractall(extract_path)
            print("  ✅ Dézippé.")
        else:
            print("  ⚡ Dossier déjà extrait.")

        # 3. Trouver le .shp principal
        shp_files = list(extract_path.glob("*.shp"))
        if not shp_files:
            print("  ❌ Pas de fichier .shp trouvé !")
            continue
        shp_path = shp_files[0]

        # 4. Charger dans GeoPandas
        try:
            gdf = gpd.read_file(shp_path)
            print("  ✅ Chargé avec GeoPandas.")
        except Exception as e:
            print(f"  ❌ Erreur lecture GeoPandas: {e}")
            continue

        # 5. Exporter en GPKG
        gdf.to_file(gpkg_path, driver="GPKG")
        print(f"  ✅ Exporté en GPKG : {gpkg_path}")

        # 6. Nettoyer : supprimer le ZIP, le dossier extrait
        try:
            os.remove(zip_path)
            print("  🗑️ ZIP supprimé.")
        except Exception:
            pass
        try:
            shutil.rmtree(extract_path)
            print("  🗑️ Dossier extrait supprimé.")
        except Exception:
            pass

    print("\n✨ Terminé ! Tous les fichiers .gpkg sont dans :", output_dir)


# Usage of function

In [12]:
my_layers = [
    ("cultural", "admin_0_countries"),
    ("physical", "ocean"),
    ("physical", "rivers_lake_centerlines"),
    ("cultural", "populated_places"),
    ("cultural", "roads"),
]

download_and_convert_natural_earth(my_layers, resolution="10m", output_dir="data_input/basemap_elements")



🔻 Traitement: admin_0_countries (cultural, 10m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_10m_admin_0_countries.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: ocean (physical, 10m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_10m_ocean.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: rivers_lake_centerlines (physical, 10m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_10m_rivers_lake_centerlines.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: populated_places (cultural, 10m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_10m_populated_places.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: roads (cultural, 10m)
  ✅ ZIP téléchargé.
 

# Get rivers from OSM

In [19]:
import geopandas as gpd
import osmnx as ox


# =============================================================================
# =============================================================================

# --- 1. Paramètres utilisateur ---
geojson_path = "myanmar.json"    # <- à adapter
# =============================================================================
# =============================================================================


In [ ]:
# import os
# import geopandas as gpd
# import osmnx as ox
# import pandas as pd
# import logging
# from typing import Optional, Tuple
# import time

# # Configuration du logging
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# def split_large_bbox(n, s, e, w, max_size=0.5):
#     """
#     Découpe une bbox trop grande en plus petites parties.
    
#     Args:
#         n, s, e, w: coordonnées de la bbox
#         max_size: taille maximale en degrés (défaut: 0.5°)
    
#     Returns:
#         Liste de tuples (n, s, e, w) pour chaque sous-bbox
#     """
#     height = n - s
#     width = e - w
    
#     # Si la bbox est assez petite, on la retourne telle quelle
#     if height <= max_size and width <= max_size:
#         return [(n, s, e, w)]
    
#     # Calcul du nombre de subdivisions nécessaires
#     rows = max(1, int(height / max_size) + 1)
#     cols = max(1, int(width / max_size) + 1)
    
#     logger.info(f"Découpage de la bbox {height:.3f}° x {width:.3f}° en {rows}x{cols} = {rows*cols} parties")
    
#     bboxes = []
#     row_height = height / rows
#     col_width = width / cols
    
#     for i in range(rows):
#         for j in range(cols):
#             bbox_s = s + i * row_height
#             bbox_n = s + (i + 1) * row_height
#             bbox_w = w + j * col_width
#             bbox_e = w + (j + 1) * col_width
            
#             bboxes.append((bbox_n, bbox_s, bbox_e, bbox_w))
    
#     return bboxes

# def query_bbox_chunked(n, s, e, w, tags, timeout=180, max_retries=3, max_bbox_size=0.5):
#     """
#     Requête OSM avec découpage automatique des grandes bbox.
    
#     Args:
#         n, s, e, w: coordonnées de la bbox
#         tags: dictionnaire des tags OSM
#         timeout: délai d'attente par chunk
#         max_retries: nombre de tentatives par chunk
#         max_bbox_size: taille maximale en degrés par chunk
    
#     Returns:
#         GeoDataFrame combiné de tous les chunks
#     """
#     # Découpage de la bbox si nécessaire
#     bbox_chunks = split_large_bbox(n, s, e, w, max_bbox_size)
    
#     all_gdfs = []
    
#     for i, (chunk_n, chunk_s, chunk_e, chunk_w) in enumerate(bbox_chunks):
#         logger.info(f"Traitement du chunk {i+1}/{len(bbox_chunks)}")
        
#         try:
#             gdf_chunk = query_bbox_robust(
#                 chunk_n, chunk_s, chunk_e, chunk_w, 
#                 tags, timeout, max_retries
#             )
            
#             if not gdf_chunk.empty:
#                 all_gdfs.append(gdf_chunk)
#                 logger.info(f"Chunk {i+1}: {len(gdf_chunk)} éléments")
#             else:
#                 logger.info(f"Chunk {i+1}: aucun élément")
                
#         except Exception as e:
#             logger.error(f"Échec du chunk {i+1}/{len(bbox_chunks)}: {e}")
#             continue
    
#     # Combinaison de tous les GeoDataFrames
#     if all_gdfs:
#         combined_gdf = gpd.GeoDataFrame(pd.concat(all_gdfs, ignore_index=True))
#         combined_gdf.crs = all_gdfs[0].crs
        
#         # Suppression des doublons potentiels
#         if 'osmid' in combined_gdf.columns:
#             combined_gdf = combined_gdf.drop_duplicates(subset=['osmid'])
        
#         logger.info(f"Total combiné: {len(combined_gdf)} éléments uniques")
#         return combined_gdf
#     else:
#         # Retourne un GeoDataFrame vide avec la bonne structure
#         return gpd.GeoDataFrame(columns=['geometry'], crs='EPSG:4326')


#     """
#     Retourne un GeoDataFrame OSM avec gestion d'erreurs robuste.
#     Compatible avec OSMnx 2.x
    
#     Args:
#         n, s, e, w: coordonnées de la bbox
#         tags: dictionnaire des tags OSM
#         timeout: délai d'attente en secondes
#         max_retries: nombre maximum de tentatives
#     """
#     # Vérification des coordonnées
#     if not all(isinstance(coord, (int, float)) for coord in [n, s, e, w]):
#         raise ValueError("Les coordonnées doivent être des nombres")
    
#     if n <= s or e <= w:
#         raise ValueError(f"Bbox invalide: n={n}, s={s}, e={e}, w={w}")
    
#     # Vérification de la taille de la bbox (pas trop grande)
#     if (n - s) > 1.0 or (e - w) > 1.0:
#         logger.warning(f"Bbox très large ({n-s:.3f}° x {e-w:.3f}°) - risque de timeout")
    
#     logger.info(f"Requête OSM: bbox=({n:.4f}, {s:.4f}, {e:.4f}, {w:.4f}), tags={tags}")
    
#     for attempt in range(max_retries):
#         try:
#             # OSMnx 2.x utilise features_from_bbox
#             if hasattr(ox, "features_from_bbox"):
#                 logger.info(f"Tentative {attempt + 1}/{max_retries} avec features_from_bbox (OSMnx 2.x)")
                
#                 # Dans OSMnx 2.x, l'ordre des paramètres a changé
#                 gdf = ox.features_from_bbox(north=n, south=s, east=e, west=w, tags=tags, timeout=timeout)
#                 logger.info(f"Succès: {len(gdf)} éléments trouvés")
#                 return gdf
                
#             # Fallback pour versions très anciennes
#             elif hasattr(ox, "geometries_from_bbox"):
#                 logger.info(f"Tentative {attempt + 1}/{max_retries} avec geometries_from_bbox")
#                 gdf = ox.geometries_from_bbox(n, s, e, w, tags=tags, timeout=timeout)
#                 logger.info(f"Succès: {len(gdf)} éléments trouvés")
#                 return gdf
            
#             else:
#                 raise ImportError("Version d'OSMnx non supportée. Mets à jour: pip install -U osmnx")
        
#         except Exception as exc:
#             logger.error(f"Tentative {attempt + 1}/{max_retries} échouée: {str(exc)}")
            
#             if attempt < max_retries - 1:
#                 wait_time = 2 ** attempt  # backoff exponentiel
#                 logger.info(f"Attente de {wait_time}s avant nouvelle tentative...")
#                 time.sleep(wait_time)
#             else:
#                 raise exc

# def get_osm_rivers_and_lakes_from_geojson(
#     geojson_path: str, 
#     output_dir: str = "data_input/basemap_elements", 
#     timeout: int = 180,
#     max_retries: int = 3,
#     major_rivers_only: bool = True,
#     major_lakes_only: bool = True
# ) -> Tuple[str, str]:
#     """
#     Télécharge les rivières et lacs OSM dans la bbox du GeoJSON.
    
#     Args:
#         geojson_path: chemin vers le fichier GeoJSON
#         output_dir: dossier de sortie
#         timeout: délai d'attente par requête
#         max_retries: nombre de tentatives par requête
#         major_rivers_only: ne prendre que les grandes rivières (fleuves)
#         major_lakes_only: ne prendre que les grands lacs
    
#     Returns:
#         Tuple des chemins vers les fichiers rivers.gpkg et lakes.gpkg
#     """
#     logger.info(f"Début du traitement: {geojson_path}")
    
#     # Vérifications préliminaires
#     if not os.path.exists(geojson_path):
#         raise FileNotFoundError(f"Fichier GeoJSON introuvable: {geojson_path}")
    
#     os.makedirs(output_dir, exist_ok=True)
    
#     try:
#         # Lecture du GeoJSON
#         logger.info("Lecture du GeoJSON...")
#         region = gpd.read_file(geojson_path)
        
#         if region.empty:
#             raise ValueError("Le GeoJSON est vide")
        
#         # Reprojection en WGS84 si nécessaire
#         if region.crs != 'EPSG:4326':
#             logger.info(f"Reprojection de {region.crs} vers EPSG:4326")
#             region = region.to_crs(4326)
        
#         # Calcul de la bbox
#         minx, miny, maxx, maxy = region.total_bounds
#         n, s, e, w = maxy, miny, maxx, minx
        
#         logger.info(f"Bbox calculée: N={n:.4f}, S={s:.4f}, E={e:.4f}, W={w:.4f}")
        
#         # Tags optimisés pour grandes rivières seulement
#         if major_rivers_only:
#             river_tags = {
#                 "waterway": ["river"]  # Seulement les rivières principales
#             }
#             logger.info("Mode: grandes rivières seulement (waterway=river)")
#         else:
#             river_tags = {"waterway": True}
#             logger.info("Mode: toutes les voies d'eau")
        
#         # Tags optimisés pour grands lacs seulement
#         if major_lakes_only:
#             lake_tags = {
#                 "natural": "water",
#                 "water": ["lake", "reservoir"]  # Seulement lacs et réservoirs
#             }
#             logger.info("Mode: grands lacs seulement (natural=water + water=lake/reservoir)")
#         else:
#             lake_tags = {"natural": "water"}
#             logger.info("Mode: tous les plans d'eau")
        
#         # Requêtes OSM avec découpage automatique - chunks plus gros pour les grandes features
#         logger.info("Téléchargement des rivières...")
#         try:
#             gdf_riv = query_bbox_chunked(n, s, e, w, tags=river_tags, 
#                                        timeout=timeout, max_retries=max_retries,
#                                        max_bbox_size=1.0)  # Chunks plus gros pour les grandes rivières
#         except Exception as e:
#             logger.error(f"Échec du téléchargement des rivières: {e}")
#             gdf_riv = gpd.GeoDataFrame(columns=['geometry'], crs='EPSG:4326')
        
#         logger.info("Téléchargement des lacs...")
#         try:
#             gdf_lak = query_bbox_chunked(n, s, e, w, tags=lake_tags, 
#                                        timeout=timeout, max_retries=max_retries,
#                                        max_bbox_size=1.0)  # Chunks plus gros pour les grands lacs
#         except Exception as e:
#             logger.error(f"Échec du téléchargement des lacs: {e}")
#             gdf_lak = gpd.GeoDataFrame(columns=['geometry'], crs='EPSG:4326')
        
#         # Filtrage des géométries avec filtres additionnels
#         logger.info("Filtrage des géométries...")
#         if not gdf_riv.empty:
#             # Filtrer par type de géométrie
#             gdf_riv = gdf_riv[gdf_riv.geometry.type.isin(["LineString", "MultiLineString"])]
            
#             # Filtrage additionnel pour les grandes rivières
#             if major_rivers_only and len(gdf_riv) > 0:
#                 initial_count = len(gdf_riv)
                
#                 # Garder seulement les rivières avec un nom (plus importantes)
#                 if 'name' in gdf_riv.columns:
#                     gdf_riv = gdf_riv[gdf_riv['name'].notna()]
#                     logger.info(f"Filtrage par nom: {initial_count} → {len(gdf_riv)} rivières")
                
#                 # Optionnel: filtrer par longueur des géométries
#                 if len(gdf_riv) > 1000:  # Si encore trop de rivières
#                     gdf_riv['length'] = gdf_riv.geometry.length
#                     # Garder les 20% les plus longues
#                     threshold = gdf_riv['length'].quantile(0.8)
#                     gdf_riv = gdf_riv[gdf_riv['length'] >= threshold]
#                     gdf_riv = gdf_riv.drop(columns=['length'])
#                     logger.info(f"Filtrage par longueur: → {len(gdf_riv)} rivières")
        
#         if not gdf_lak.empty:
#             # Filtrer par type de géométrie
#             gdf_lak = gdf_lak[gdf_lak.geometry.type.isin(["Polygon", "MultiPolygon"])]
            
#             # Filtrage additionnel pour les grands lacs
#             if major_lakes_only and len(gdf_lak) > 0:
#                 initial_count = len(gdf_lak)
                
#                 # Garder seulement les lacs avec un nom (plus importants)
#                 if 'name' in gdf_lak.columns:
#                     gdf_lak = gdf_lak[gdf_lak['name'].notna()]
#                     logger.info(f"Filtrage par nom: {initial_count} → {len(gdf_lak)} lacs")
                
#                 # Optionnel: filtrer par superficie
#                 if len(gdf_lak) > 500:  # Si encore trop de lacs
#                     gdf_lak['area'] = gdf_lak.geometry.area
#                     # Garder les 30% les plus grands
#                     threshold = gdf_lak['area'].quantile(0.7)
#                     gdf_lak = gdf_lak[gdf_lak['area'] >= threshold]
#                     gdf_lak = gdf_lak.drop(columns=['area'])
#                     logger.info(f"Filtrage par superficie: → {len(gdf_lak)} lacs")
        
#         # Export des fichiers
#         p_riv = os.path.join(output_dir, "osm_rivers.gpkg")
#         p_lak = os.path.join(output_dir, "osm_lakes.gpkg")
        
#         logger.info(f"Export des rivières: {len(gdf_riv)} éléments")
#         gdf_riv.to_file(p_riv, driver="GPKG")
        
#         logger.info(f"Export des lacs: {len(gdf_lak)} éléments")
#         gdf_lak.to_file(p_lak, driver="GPKG")
        
#         logger.info(f"✅ Exporté avec succès:\n  • {p_riv}\n  • {p_lak}")
#         return p_riv, p_lak
        
#     except Exception as e:
#         logger.error(f"Erreur fatale: {e}")
#         raise

# # Fonction d'aide pour diagnostiquer les problèmes
# def diagnose_osm_setup():
#     """Diagnostique la configuration OSM/OSMnx"""
#     print("=== DIAGNOSTIC OSM/OSMnx ===")
    
#     try:
#         import osmnx as ox
#         print(f"✅ OSMnx version: {ox.__version__}")
        
#         # Test des fonctions disponibles
#         if hasattr(ox, "features_from_bbox"):
#             print("✅ features_from_bbox disponible")
#         elif hasattr(ox, "geometries_from_bbox"):
#             print("✅ geometries_from_bbox disponible")
#         else:
#             print("❌ Aucune fonction de requête trouvée")
        
#         # Test de configuration pour OSMnx 2.x
#         try:
#             # Dans OSMnx 2.x, pas de ox.settings.timeout
#             print("Configuration OSMnx 2.x détectée")
            
#             # Test simple pour voir si ça fonctionne
#             print("Test de requête simple...")
#             # Petite bbox pour test rapide (autour de Paris)
#             test_gdf = ox.features_from_bbox(north=48.86, south=48.85, east=2.35, west=2.34, 
#                                            tags={"amenity": "cafe"}, timeout=30)
#             print(f"✅ Test réussi: {len(test_gdf)} éléments trouvés")
            
#         except Exception as e:
#             print(f"⚠️  Erreur lors du test: {e}")
        
#     except ImportError as e:
#         print(f"❌ Erreur d'import OSMnx: {e}")
    
#     try:
#         import geopandas as gpd
#         print(f"✅ GeoPandas version: {gpd.__version__}")
#     except ImportError as e:
#         print(f"❌ Erreur d'import GeoPandas: {e}")

# # Exemple d'utilisation avec gestion d'erreurs
# if __name__ == "__main__":
#     # Diagnostic
#     diagnose_osm_setup()
    
#     # Test de la fonction - MODE OPTIMISÉ pour grandes rivières seulement
#     try:
#         geojson_path = geojson_path  # Remplace par ton fichier
#         rivers_file, lakes_file = get_osm_rivers_and_lakes_from_geojson(
#             geojson_path, 
#             timeout=120,  # 2 minutes par chunk
#             max_retries=2,
#             major_rivers_only=True,  # SEULEMENT LES GRANDES RIVIÈRES
#             major_lakes_only=True    # SEULEMENT LES GRANDS LACS
#         )
#         print(f"Succès! Fichiers créés: {rivers_file}, {lakes_file}")
#     except Exception as e:
#         print(f"Échec: {e}")

INFO:__main__:Début du traitement: myanmar.json
INFO:__main__:Lecture du GeoJSON...
INFO:__main__:Bbox calculée: N=33.1994, S=12.6942, E=105.7845, W=83.7163
INFO:__main__:Mode: grandes rivières seulement (waterway=river)
INFO:__main__:Mode: grands lacs seulement (natural=water + water=lake/reservoir)
INFO:__main__:Téléchargement des rivières...
INFO:__main__:Découpage de la bbox 20.505° x 22.068° en 21x23 = 483 parties
INFO:__main__:Traitement du chunk 1/483
INFO:__main__:Requête OSM: bbox=(13.6707, 12.6942, 84.6758, 83.7163), tags={'waterway': ['river']}
INFO:__main__:Tentative 1/2 avec features_from_bbox (OSMnx 2.x)
ERROR:__main__:Tentative 1/2 échouée: features_from_bbox() got multiple values for argument 'tags'
INFO:__main__:Attente de 1s avant nouvelle tentative...


=== DIAGNOSTIC OSM/OSMnx ===
✅ OSMnx version: 2.0.5
✅ features_from_bbox disponible
Configuration OSMnx 2.x détectée
Test de requête simple...
⚠️  Erreur lors du test: features_from_bbox() got an unexpected keyword argument 'north'
✅ GeoPandas version: 1.1.1


INFO:__main__:Tentative 2/2 avec features_from_bbox (OSMnx 2.x)
ERROR:__main__:Tentative 2/2 échouée: cannot access local variable 'e' where it is not associated with a value
ERROR:__main__:Échec du chunk 1/483: cannot access local variable 'e' where it is not associated with a value
INFO:__main__:Traitement du chunk 2/483
INFO:__main__:Requête OSM: bbox=(13.6707, 12.6942, 85.6353, 84.6758), tags={'waterway': ['river']}
INFO:__main__:Tentative 1/2 avec features_from_bbox (OSMnx 2.x)
ERROR:__main__:Tentative 1/2 échouée: features_from_bbox() got multiple values for argument 'tags'
INFO:__main__:Attente de 1s avant nouvelle tentative...
INFO:__main__:Tentative 2/2 avec features_from_bbox (OSMnx 2.x)
ERROR:__main__:Tentative 2/2 échouée: cannot access local variable 'e' where it is not associated with a value
ERROR:__main__:Échec du chunk 2/483: cannot access local variable 'e' where it is not associated with a value
INFO:__main__:Traitement du chunk 3/483
INFO:__main__:Requête OSM: bbox=

Succès! Fichiers créés: data_input/basemap_elements/osm_rivers.gpkg, data_input/basemap_elements/osm_lakes.gpkg
